# Simulator

In [1]:
import sys
sys.path.insert(0, "..")
import pandas as pd
import matplotlib.pyplot as plt
from trading_simulator import TradingSimulator, DLSimulator
from dl_solutions.dlsolutions import CryptoDLSolutions
from data_processor.data_processing import DataProcessor

Using TensorFlow backend.


## Classic

In [ ]:
cryptos = ['ADA', 'BTC', 'ETH', 'LTC', 'LNK']
sel = 0

In [ ]:
for crypto in cryptos:
    processor = DataProcessor([crypto])
    sim = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2, log = False)
    sim.simulate()

In [ ]:
processor = DataProcessor([crypto])
sim1 = TradingSimulator(processor, crypto, strategy = [1], loss_allowed = 0.2)
sim2 = TradingSimulator(processor, crypto, strategy = [2], loss_allowed = 0.2)
sim3 = TradingSimulator(processor, crypto, strategy = [3], loss_allowed = 0.2)
sim4 = TradingSimulator(processor, crypto, strategy = [4], loss_allowed = 0.2)
sim5 = TradingSimulator(processor, crypto, strategy = [5], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [6], loss_allowed = 0.2)
sim6 = TradingSimulator(processor, crypto, strategy = [7], loss_allowed = 0.2)
sim7 = TradingSimulator(processor, crypto, strategy = [1, 2, 3, 4, 5, 6, 7], loss_allowed = 0.2)

In [ ]:
sim1.simulate()

In [ ]:
sim2.simulate()

In [ ]:
sim3.simulate()

In [ ]:
sim4.simulate()

In [ ]:
sim5.simulate()

In [ ]:
sim6.simulate()

In [ ]:
sim7.simulate()

## DL Solution

In [34]:
crypto = 'ETH'
periods_to_retraining = 1000
prev_periods = 5
pred_periods = 10
model_selector = 'lstm'
#columns = ['close']
#columns = ['weekday_num', 'RSI', 'MACD', 'close']
#columns = ['RSI', 'MACD', 'close']
#columns = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun', 'RSI', 'MACD', 'close']
columns = ['SO_K', 'SMA 50', 'SMA 200', 'close']
num_features = len(columns)

processor = DataProcessor([crypto])
sim = DLSimulator(processor, crypto, periods_to_retraining, prev_periods, pred_periods, model_selector, columns, None)
# TODO Add target if we want features as timestamps
#sim.simulate()
df = sim.get_df()
real_df = df.loc[327:1577]
real_df = df.loc[327:1577]
to_pred_df = df.loc[1578:1578]
real_and_pred = df.iloc[:1001]
real_df

Loading... ETH
Extracting columns columns for ETH
Proccessing and arranging columns for LSTM model


,close_4,SMA 50_4,SMA 200_4,SO_K_4,close_3,SMA 50_3,SMA 200_3,SO_K_3,close_2,SMA 50_2,...,SMA 200_1,SO_K_1,close_0,SMA 50_0,SMA 200_0,SO_K_0,close,SMA 50,SMA 200,SO_K
327,467.87,542.7508,710.05045,51.976058,470.47,538.8078,708.48280,79.987879,485.57,534.6672,...,705.40345,89.934426,471.71,525.5736,703.84995,73.475410,468.57,498.6578,691.61595,52.400284
328,470.47,538.8078,708.48280,79.987879,485.57,534.6672,706.92065,97.102143,486.77,530.4626,...,703.84995,73.475410,432.48,520.2892,702.83250,30.601093,449.04,496.0824,690.08620,32.578910
329,485.57,534.6672,706.92065,97.102143,486.77,530.4626,705.40345,89.934426,471.71,525.5736,...,702.83250,30.601093,444.99,516.3692,701.85575,44.273224,460.23,493.7070,688.11095,43.935857
330,486.77,530.4626,705.40345,89.934426,471.71,525.5736,703.84995,73.475410,432.48,520.2892,...,701.85575,44.273224,431.14,513.4482,700.72145,29.136612,457.47,491.0394,685.72815,41.134680
331,471.71,525.5736,703.84995,73.475410,432.48,520.2892,702.83250,30.601093,444.99,516.3692,...,700.72145,29.136612,432.46,510.0538,699.30900,19.635628,450.05,487.6472,683.27840,33.603978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573,4511.21,4306.0112,3161.34335,69.097314,4215.73,4314.5094,3166.00945,34.967369,4117.25,4319.4746,...,3178.50795,54.331135,4347.59,4336.8190,3186.40280,66.185386,3958.11,4341.8730,3264.92580,39.474461
1574,4215.73,4314.5094,3166.00945,34.967369,4117.25,4319.4746,3169.72035,48.120495,4196.44,4326.8042,...,3186.40280,66.185386,4306.40,4348.0478,3195.77990,62.954975,3875.42,4333.6610,3270.77215,37.766174
1575,4117.25,4319.4746,3169.72035,48.120495,4196.44,4326.8042,3178.50795,54.331135,4347.59,4336.8190,...,3195.77990,62.954975,4436.91,4359.2454,3206.49010,73.190492,3959.93,4324.4818,3277.39895,37.452474
1576,4196.44,4326.8042,3178.50795,54.331135,4347.59,4336.8190,3186.40280,66.185386,4306.40,4348.0478,...,3206.49010,73.190492,4105.64,4358.1352,3216.53635,47.209957,3924.14,4316.5110,3283.48855,33.213309


In [35]:
norm_strat = 2
strat = 1
layers = 4
batch_size = 64
epochs = 150
num_timestamps = prev_periods

lstm = CryptoDLSolutions(real_df, norm_strat, strat, layers, batch_size, epochs, num_timestamps, num_features)
lstm.build()
print(lstm.get_model().summary())
lstm.train()

Input shape: (1251, 5, 4) (1251, 4) (0, 5, 4) (0, 4)
[[[0.08130654 0.10085752 0.18641527 0.51553516]
  [0.0818569  0.09965788 0.18532927 0.79838313]
  [0.08505319 0.09841874 0.18417156 0.97119357]
  [0.0853072  0.09742758 0.18307606 0.8988179 ]
  [0.08211938 0.09627508 0.18220036 0.7326238 ]]

 [[0.0818569  0.09992308 0.18590023 0.79838313]
  [0.08505319 0.09867922 0.18481762 0.97119357]
  [0.0853072  0.09742758 0.18367636 0.8988179 ]
  [0.08211938 0.09627508 0.18257067 0.7326238 ]
  [0.07381536 0.09502937 0.18187003 0.29970251]]

 [[0.08505319 0.09894181 0.185387   0.97119357]
  [0.0853072  0.09768543 0.18432069 0.8988179 ]
  [0.08211938 0.09627508 0.18316932 0.7326238 ]
  [0.07381536 0.09502937 0.18223968 0.29970251]
  [0.07646342 0.09410529 0.18155292 0.43775618]]

 ...

 [[0.85378962 0.99588961 0.99451912 0.47660381]
  [0.87055218 0.99497892 0.99434287 0.53931543]
  [0.90254688 0.99471335 0.99344375 0.65901315]
  [0.89382798 0.99736035 0.99324749 0.62639423]
  [0.9214537  1.       

In [38]:
pred = lstm.predict()
print('pred:', pred)

testx (1, 5, 4)
preds [[0.8454378 0.8920391 0.8771051 0.3571905]]
preds rev norm [[4077.79428033 3901.26690534 2902.26450125   36.294409  ]]
real [[4014.15       4303.5558     3295.54915      43.87458988]]
Test RMSE: 283.115
pred: [[4077.79428033 3901.26690534 2902.26450125   36.294409  ]]


In [36]:
to_pred_df = df.loc[1578:1578]
to_pred_df

,close_4,SMA 50_4,SMA 200_4,SO_K_4,close_3,SMA 50_3,SMA 200_3,SO_K_3,close_2,SMA 50_2,...,SMA 200_1,SO_K_1,close_0,SMA 50_0,SMA 200_0,SO_K_0,close,SMA 50,SMA 200,SO_K
1578,4306.4,4348.0478,3195.7799,62.954975,4436.91,4359.2454,3206.4901,73.190492,4105.64,4358.1352,...,3222.7839,30.920655,4089.37,4357.3776,3229.7052,45.933949,4014.15,4303.5558,3295.54915,43.87459


In [37]:
lstm.set_test(to_pred_df)

In [6]:
df[-20:]

,close_1,MACD_1,RSI_1,weekday_num_1,close_0,MACD_0,RSI_0,weekday_num_0,close,MACD,RSI,weekday_num
1569,4582.95,36.473333,55.716077,2.0,4511.21,43.536946,53.835840,3,4130.00,-57.803905,46.707531,6.0
1570,4511.21,43.536946,53.835840,3.0,4215.73,25.003914,46.826530,4,3781.93,-87.109146,39.777252,0.0
1571,4215.73,25.003914,46.826530,4.0,4117.25,2.342833,44.735958,5,3858.99,-102.929159,41.834910,1.0
1572,4117.25,2.342833,44.735958,5.0,4196.44,-9.121124,46.793026,6,4019.08,-101.380062,45.965427,2.0
1573,4196.44,-9.121124,46.793026,6.0,4347.59,-5.941362,50.574650,0,3958.11,-103.874758,44.664534,3.0
1574,4347.59,-5.941362,50.574650,0.0,4306.40,-6.668199,49.541304,1,3875.42,-111.241886,42.891556,4.0
1575,4306.40,-6.668199,49.541304,1.0,4436.91,3.249386,52.829940,2,3959.93,-109.004601,45.282169,5.0
1576,4436.91,3.249386,52.829940,2.0,4105.64,-15.443552,44.841174,3,3924.14,-108.864567,44.433846,6.0
1577,4105.64,-15.443552,44.841174,3.0,3897.94,-46.481683,40.686916,4,3944.73,-105.871725,45.071392,0.0
1578,3897.94,-46.481683,40.686916,4.0,4089.37,-54.998832,45.681743,5,4014.15,-96.782613,47.268176,1.0


In [ ]:
to_pred_df = pd.read_csv('/Users/gonzalozabala/Documents/Master IA/AIMaster/TFM/Code/crypto_bot/Datasets/ETH.csv', header=[1])
to_pred_df = pd.DataFrame(to_pred_df['close'].iloc[:10]).T
to_pred_df['y'] = 686
to_pred_df

In [ ]:
processor = DataProcessor(['ETH'])
processor.load_data()
processor.clean_data('ETH')
#processor.feature_extraction('ETH')
#columns = ['close', 'Open Close Difference', 'Volume USDT' ,'Result']
#columns = ['close']
#df = processor.feature_selection('ETH', columns) 
df.loc[1588]